1. używamy macierzy rzadkich, bo mamy ogromne macierze
2. ~ 100k artykułów z wikipedii
3. normalizacja po długości tekstu
4. artykuły jako bag of words
5. najpierw używamy prostej wyszukiwarki, używajac kąty miedzy wektorami jako miary podobieństwa

6. potem uzyjemy SVD do przyblizenia tych wektorow przez obnizenie wektorowości (mniejszy zbiór)

uzyc NLP zeby zredukowac ilosc słów (przyimki liczby mnogie etc)
STOP WORDs

EXTRA:
Latent Dirichlet Alocation
Bag of topics
BERT


NAPISAĆ GUI - front ma działać szybko

## Loading dataset

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re
import tensorflow_datasets as tfds
from sklearn.feature_extraction import DictVectorizer
from collections import Counter
from scipy.sparse import find, csr_matrix, linalg, diags

In [24]:
def prepare_data():
    # loding dataset
    ds = tfds.load('wikipedia/20201201.simple', shuffle_files=False, split="train")
    ds_np = tfds.as_numpy(ds)
    
    # spliting articles into lists of words and preparing raw texts for quick querying
    arts_split = []
    texts = []
    for art in ds_np:
        words = re.sub(r"[^a-zA-Z0-9]", " ", art['text'].decode("utf-8").lower()).split(' ')
        arts_split.append(words)
        texts.append(art['title'].decode("utf-8") + "\n\n" + art['text'].decode("utf-8"))
    
    # freeing memory  
    del ds
    del ds_np
    
    # building sparse matrix
    vectorizer = DictVectorizer(sparse=True)
    doc_term_mat = vectorizer.fit_transform(Counter(art) for art in arts_split)
    
    # multiplying by IDF
    IDF = np.log(doc_term_mat.shape[1] / (doc_term_mat != 0).sum(axis = 0))
    doc_term_mat_processed = doc_term_mat.multiply(IDF)
    
    # using svd and low rank approximation to decrease noise
# NOTE WE"RE NOT USING IT AS MULTIPYING THOSE MATRICIES AT FULL DATASET TAKES AGES
#     u, s, vh = linalg.svds(doc_term_mat_processed, k=10)

#     u = csr_matrix(u)
#     s = csr_matrix(diags(s))
#     vh = csr_matrix(vh)

#     doc_term_mat_processed = u @ s @ vh
    
    return doc_term_mat, doc_term_mat_processed, vectorizer, texts

In [25]:
def make_query(doc_term_mat, vectorizer, texts, query_string, no_of_results = 5):
    # split query string into words and vectorize it
    query_words = re.sub(r"[^a-zA-Z0-9]", " ", query_string.lower()).split(' ')
    query_vector = vectorizer.transform(Counter(query_words))
    
    # computer correlations
    corr = doc_term_mat @ query_vector.transpose() / linalg.norm(query_vector) / linalg.norm(doc_term_mat)
    
    # find best results
    result = find(corr)

    idx = np.argpartition(result[2], -no_of_results)[-no_of_results:]
    indices = idx[np.argsort((-result[2])[idx])]
    
    # print teasers
    
    for ind in indices:
        print(texts[result[0][ind]][:100])
        print("\n-----------------------------------\n")

In [26]:
doc_term_mat, doc_term_mat_processed, vectorizer, texts = prepare_data()

In [27]:
make_query(doc_term_mat, vectorizer, texts, "tiger")

Tiger shark

The tiger shark (Galeocerdo cuvier) is a species of requiem shark, and is the only memb

-----------------------------------

Tiger

The tiger (Panthera tigris) is the largest living member of the cat family, the Felidae. It f

-----------------------------------

South China tiger

The South China tiger (Panthera tigris amoyensis) is an almost extinct subspecies

-----------------------------------

Drunken Tiger

Drunken Tiger (DT) is a South Korean hip hop group which issued their first album in 

-----------------------------------

Tipu's Tiger

Tipu's Tiger (a.k.a. Tippoo's Tiger) is an 18th century moving model, musical instrume

-----------------------------------



In [28]:
make_query(doc_term_mat_processed, vectorizer, texts, "tiger")

Tiger shark

The tiger shark (Galeocerdo cuvier) is a species of requiem shark, and is the only memb

-----------------------------------

Tiger

The tiger (Panthera tigris) is the largest living member of the cat family, the Felidae. It f

-----------------------------------

South China tiger

The South China tiger (Panthera tigris amoyensis) is an almost extinct subspecies

-----------------------------------

Drunken Tiger

Drunken Tiger (DT) is a South Korean hip hop group which issued their first album in 

-----------------------------------

Tipu's Tiger

Tipu's Tiger (a.k.a. Tippoo's Tiger) is an 18th century moving model, musical instrume

-----------------------------------



In [32]:
make_query(doc_term_mat, vectorizer, texts, "fanta sweet drink")

Hot chocolate

Hot chocolate (also called hot cocoa, drinking chocolate or just cocoa) is a hot drin

-----------------------------------

Fanta

Fanta is an orange soda drink. It was created in Germany in 1940. Fanta was created during Wo

-----------------------------------

Coffee

Coffee is a plant (Coffea) and the name of the drink that is made from this plant. The coffe

-----------------------------------

Alcoholism

Alcoholism is addiction to alcohol. People who have alcoholism are called alcoholics.  


-----------------------------------

Sweet potato

The sweet potato (Ipomoea batatas) is a plant that is a member of the morning glory fa

-----------------------------------



In [31]:
make_query(doc_term_mat_processed, vectorizer, texts, "fanta sweet drink")

Fanta

Fanta is an orange soda drink. It was created in Germany in 1940. Fanta was created during Wo

-----------------------------------

Hot chocolate

Hot chocolate (also called hot cocoa, drinking chocolate or just cocoa) is a hot drin

-----------------------------------

Coffee

Coffee is a plant (Coffea) and the name of the drink that is made from this plant. The coffe

-----------------------------------

Alcoholism

Alcoholism is addiction to alcohol. People who have alcoholism are called alcoholics.  


-----------------------------------

Sweet potato

The sweet potato (Ipomoea batatas) is a plant that is a member of the morning glory fa

-----------------------------------



In [38]:
make_query(doc_term_mat_processed, vectorizer, texts, "web development internet technology")

Internet Explorer

Internet Explorer (IE); formerly known as Microsoft Internet Explorer and Windows

-----------------------------------

Asian Masters Athletics Championships

The Asian Masters Athletics Championships are the biannual ch

-----------------------------------

Tim Berners-Lee

Sir Timothy John "Tim" Berners-Lee OM KBE FRS (born 8 June 1955) is the inventor of

-----------------------------------

Development

Development  has meaning in several contexts:

Engineering
 Land improvement and/or con

-----------------------------------

Norton Internet Security

Norton Internet Security is a security program made by Symantec Corporatio

-----------------------------------



In [40]:
make_query(doc_term_mat_processed, vectorizer, texts, "software development internet technology")

Internet Explorer

Internet Explorer (IE); formerly known as Microsoft Internet Explorer and Windows

-----------------------------------

Development

Development  has meaning in several contexts:

Engineering
 Land improvement and/or con

-----------------------------------

Norton Internet Security

Norton Internet Security is a security program made by Symantec Corporatio

-----------------------------------

Cloud computing

Cloud computing is when computing services are provided by a company or place outsi

-----------------------------------

Free Software Foundation

The Free Software Foundation (FSF) is a non-profit corporation founded by 

-----------------------------------

